In [1]:
# 爬蟲類套件
import requests
import json
from bs4 import BeautifulSoup
# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
from tomorrow import threads
# 資料整理類套件
import pandas as pd
import re
from datetime import datetime
from pymongo import MongoClient
import pymongo

In [59]:
#取得網站內所有要爬蟲的連結
def GetAllLinks():
    #網域名稱
    domain="https://www.chinatimes.com"     #自定義
    #初始設定j為第1頁
    j=1
    #data內容是任意字串皆可，只要不是空字串都可以
    data = "length of data"                 #自定義
    links = []
    while len(data) !=0:
        #網站為分頁式
        #先取得每個分頁的網址
        page_url = "https://www.chinatimes.com/politic/total?page=" + str(j)     #自定義
        res = requests.get(page_url, headers = headers)
        soup = BeautifulSoup(res.text,"html.parser")
        
        #算出每一頁網址內的資料筆數存為變數data，在上面while迴圈中只要data長度不為0，迴圈就會一直跑
        data = soup.find_all("ul", class_="vertical-list list-style-none")[0].find_all("li")   #自定義  
        #在每一頁之中，找出每篇文章的網址，並且append到名為links的list
        for i in soup.find_all('h3'):       #自定義
            url = i.find('a')['href']
            links.append(domain+url)
        #爬完一頁後，j+1繼續爬取下一頁的網址
        j+=1
        
    return links

#測試網址抓取
#GetAllLinks()

['https://www.chinatimes.com/realtimenews/20191120003058-260407',
 'https://www.chinatimes.com/realtimenews/20191120003111-260407',
 'https://www.chinatimes.com/realtimenews/20191120003000-260407',
 'https://www.chinatimes.com/realtimenews/20191120002994-260407',
 'https://www.chinatimes.com/realtimenews/20191120003061-260407',
 'https://www.chinatimes.com/realtimenews/20191120003044-260407',
 'https://www.chinatimes.com/realtimenews/20191120002982-260407',
 'https://www.chinatimes.com/realtimenews/20191120002628-260407',
 'https://www.chinatimes.com/realtimenews/20191120002915-260407',
 'https://www.chinatimes.com/realtimenews/20191120002878-260407',
 'https://www.chinatimes.com/realtimenews/20191120002868-260407',
 'https://www.chinatimes.com/realtimenews/20191120002836-260407',
 'https://www.chinatimes.com/realtimenews/20191120002845-260407',
 'https://www.chinatimes.com/realtimenews/20191120002813-260407',
 'https://www.chinatimes.com/realtimenews/20191120002733-260407',
 'https://

In [9]:
#取得網站內所有要爬蟲的連結
def GetLinks_Keyword(keyword,pages):
    #搜尋網址基本連結(不含關鍵字)
    domain="https://www.chinatimes.com/search/" + str(keyword)     #自定義
    #初始設定j為第1頁
    pages = int(pages)
    j=1
    #data內容是任意字串皆可，只要不是空字串都可以
    data = "length of data"                 #自定義
    links = []
    while len(data) !=0:
        #網站為分頁式
        #先取得每個分頁的網址
        page_url = "https://www.chinatimes.com/search/" + str(keyword) + "?page=" + str(j)     #自定義
        res = requests.get(page_url, headers = headers)
        soup = BeautifulSoup(res.text,"html.parser")
        
        #算出每一頁網址內的資料筆數存為變數data，在上面while迴圈中只要data長度不為0，迴圈就會一直跑
        data = soup.find_all("ul", class_="vertical-list list-style-none")[0].find_all("li")   #自定義  
        #在每一頁之中，找出每篇文章的網址，並且append到名為links的list
        for i in soup.find_all('h3'):       #自定義
            url = i.find('a')['href']
            links.append(url)
        #爬完一頁後，j+1繼續爬取下一頁的網址
        j+=1
        #頁數太多時可自行設定要停止的頁數j
        if j > pages:
            break
        
    return links

#測試網址抓取關鍵字為政治，頁數為15頁
GetLinks_Keyword("政治",15)

300


In [4]:
#爬取每篇文章的內容
def GetNews_chinatimes(response):      #自定義
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']    #自定義
    ndf = pd.DataFrame(data = [{'Source':'News',          #自定義
                                'Site':'Chinatimes',
                                'Url':soup.find('meta', {'property':'og:url'})['content'],
                                'Author':soup.find('div', attrs={'class':'author'}).text.strip(),
                                'Title':soup.find('h1', attrs={'class':'article-title'}).text,
                                'CreateTime':datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+08:00'),
                                'Content':'\n'.join(i.text for i in soup.find('div',attrs={'class':'article-body'}).find_all('p')),
                                'Tag':soup.find('meta',{'name':'news_keywords'})['content']}],
                       columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Tag']) 
    return ndf

In [60]:
#測試抓取一篇文章並存為是否成功
testurl = 'https://www.chinatimes.com/newspapers/20191028000424-260118?chdtv'   #自定義
resp = requests.get(testurl)
GetNews_chinatimes(resp)

,Source,Site,Url,CreateTime,Author,Title,Content,Tag
0,News,Chinatimes,https://www.chinatimes.com/newspapers/20191028...,2019-10-28 04:10:19,周毓翔,直球對決 韓接招青年辣提問！可以討厭我 但請聽我的政策,國民黨總統參選人韓國瑜昨晚在台北舉辦青年論壇，標榜「走出同溫層」，果然在場青年提問辛辣，毫不...,"韓國瑜,國民黨,2020大選"


In [61]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [62]:
links = GetAllLinks() 

def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetNews_chinatimes(response)
            allData.append(ndf)
        except:
            pass
    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

#爬完的內容存成DataFrame df
df = CrawlAllNews(links)

There are 200 links in pages.
There are 198 News in DataFrame.
CPU times: user 13.2 s, sys: 1.52 s, total: 14.7 s
Wall time: 10.9 s


In [55]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

        


CPU times: user 39.9 ms, sys: 6.32 ms, total: 46.3 ms
Wall time: 477 ms


In [ ]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

In [ ]:
dataframe_to_mongo_test(df)